In [1]:
# Mocking data for dev

In [2]:
import json
import pandas as pd
from faker import Faker

In [3]:
post_code_df = pd.read_csv('_62_postcode.csv').drop_duplicates(subset=['จังหวัด','อำเภอ'])
electoral_district_df = pd.read_csv('_62_electzone_processed.csv', index_col=0)

In [4]:
electoral_district_df

,จังหวัด,จำนวนสมาชิกสภาผู้แทนราษฎร,จำนวนเขตเลือกตั้ง,ท้องที่ที่ประกอบเป็นเขตเลือกตั้ง,ลำดับที่,เขตเลือกตั้งที่,interior,exterior,subinterior
0,ตราด,1,1,ทั้งจังหวัด,1,1,NaN,NaN,NaN
1,นครนายก,1,1,ทั้งจังหวัด,2,1,NaN,NaN,NaN
2,พังงา,1,1,ทั้งจังหวัด,3,1,NaN,NaN,NaN
3,แม่ฮ่องสอน,1,1,ทั้งจังหวัด,4,1,NaN,NaN,NaN
4,ระนอง,1,1,ทั้งจังหวัด,5,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1754,กรุงเทพมหานคร,30,30,ภาษีเจริญ,77,29,NaN,NaN,NaN
1755,กรุงเทพมหานคร,30,30,ตลิ่งชัน,77,29,NaN,ตลิ่งชัน,NaN
1756,กรุงเทพมหานคร,30,30,ตลิ่งชัน,77,29,NaN,ฉิมพลี,NaN
1757,กรุงเทพมหานคร,30,30,บางพลัด,77,30,NaN,NaN,NaN


In [5]:
district_list = list()
electdistrict = dict()
for g, g_df in electoral_district_df.groupby(['จังหวัด', 'ท้องที่ที่ประกอบเป็นเขตเลือกตั้ง']):
    province, district = g
    electoral_nums = g_df['เขตเลือกตั้งที่'].drop_duplicates().astype(int)
    districts = g_df['ท้องที่ที่ประกอบเป็นเขตเลือกตั้ง'].drop_duplicates().values
    if 'ทั้งจังหวัด' in districts:
        districts = post_code_df[post_code_df['จังหวัด'] == province]['อำเภอ'].values
    dks = [f'{province}-{el_n}' for el_n in electoral_nums]
    for dk, en in zip(dks, electoral_nums):
        d = electdistrict.get(dk, dict(district_list=list(), province=province, electoralDistrictNumber=en))
        d['district_list'] += districts.tolist()
        electdistrict[dk] = d
    for d in districts:
        district_list.append(
            dict(district=d, province=province, electoral=[dict(fk=fk) for fk in dks]))

In [6]:
with open('district_province_list.json', 'w') as fp:
    json.dump(district_list, fp, ensure_ascii=False)

In [7]:
with open('electoral_district_table.json', 'w') as fp:
    json.dump(electdistrict, fp, ensure_ascii=False)

In [8]:
import random
import re
fake = Faker('th')

In [9]:
twfu_people=pd.read_csv('./_People_table_exported.csv')
twfu_parties=pd.read_csv('./_Parties_table_exported.csv')

In [55]:
twfu_parties = twfu_parties[(twfu_parties.PartyType == 'พรรค') & (~twfu_parties.Images.isna())]

In [56]:
image_urls = twfu_people[~twfu_people.Images.isna()].Images.apply(lambda x: re.sub('.*\((.*)\)','\\1',x)).tolist()
occupations = twfu_people[~twfu_people.ExOccupation0.isna()].ExOccupation0.unique().astype(str).tolist()

In [57]:
set([type(i) for i in image_urls])

{str}

In [67]:
people_list = []
for (province, electoral_nums), g_df in electoral_district_df.groupby(['จังหวัด', 'เขตเลือกตั้งที่']):
    is_in_province = twfu_people['MpProvince'] == province
    total_people=random.randint(10, 30)
    parties = twfu_parties.sample(total_people)['Name'].to_list()
    for i in range(1, total_people+1):
        person = dict(
            Name=random.choice(twfu_people.Name.tolist()),
            Birthdate=fake.iso8601().split('T')[0],
            Number=i,
            Education=random.choice(['ปริญญาโท', 'ปริญญาเอก', 'ปริญญาจัตวา', 'ด้อกเต้อ']),
            ExOccupation=random.choice(occupations[:5] + ['สมาชิกสภาผู้แทนราษฎร']),
            Party=parties.pop(),
            Province=province,
            Zone=int(electoral_nums),
            Image=(random.choice(image_urls) if random.random() < 0.7 else '')
        )
        people_list.append(person)

In [70]:
party_list = []
total_party = (twfu_parties.PartyType == 'พรรค').sum()
party_number = list(range(1, total_party+1))
random.shuffle(party_number)
for i,party in twfu_parties[twfu_parties.PartyType == 'พรรค'].iterrows():
    candiate_temp = []
    for i in range(random.randint(1,3)):
        candiate_temp.append(dict(Name=fake.name(), Image=random.choice(image_urls)))
    party_list.append(
        dict(Number=party_number.pop(),
             Image=re.sub('.*\((.*)\)','\\1', party.Images) if isinstance(party.Images, str) else '',
             Name=party.Name,
             PartyGroup=random.choice(['ฝ่ายค้าน', 'ฝ่ายรัฐบาล', 'ทั้งฝ่ายรัฐบาลและฝ่ายค้าน']),
             Candidate=candiate_temp,
             PartyList=[dict(Name=fake.name()) for _ in range(100)],
             Policy='',
             Promise='',
             Law='',
             Others='',
             Website=party.Website if isinstance(party.Website, str) else ''
            ))

In [71]:
with open('people_list.json', 'w') as fp:
    json.dump(people_list, fp, ensure_ascii=False)
with open('parties.json', 'w') as fp:
    json.dump(party_list, fp, ensure_ascii=False)